In [1]:
import pandas as pd
import numpy as np
import sys
import seaborn as sns
import os.path
import time, itertools, re
from sklearn.metrics import make_scorer, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline, Pipeline
from FigPlotting import clusterPlot, plot_dendrogram
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
print(os.getcwd())
fileloc_data='/'.join(os.getcwd().split('/')[0:5])+ '/data/annonymizedDatasets/'
savetag='pred_lavSQ_MHC'
print(fileloc_data)
code_path='/'.join(os.getcwd().split('/')[0:4])+'/sklvq/'
sys.path.append(code_path)
from sklvq import GMLVQ, LGMLVQ
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import (LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysis,)
from sklearn.svm import SVC

/home/sreejita/ProjectsPy/GGZ/code_scripts/ED-profiling
/home/sreejita/ProjectsPy/GGZ/data/annonymizedDatasets/


In [2]:
colsLAV=['Lav-Neg_Waardering', 'Lav-Gebrek_Vertrouwdheid', 'Lav-Alg_Ontevredenheid']
colsSQ48=['SQ48-Vijandigheid','SQ48-Agorafobie','SQ48-Angst','SQ48-Depressie', 'SQ48-Cognitieve_Klachten',
          'SQ48-Somatische_Klachten', 'SQ48-Sociale_Fobie','SQ48-Vitaliteit_Optimisme', 'SQ48-Werk_Studie']
colsMHCSF=['MHCSF-EmotionWB', 'MHCSF-SocialWB', 'MHCSF-PsychWB']
cols2consider=['Main-Age','Main-ED_Codes', 'EDEQ-Score']+colsLAV+colsSQ48+colsMHCSF
cols2train=['Main-Age']+colsLAV+colsSQ48+colsMHCSF
colNames=['Main-Age', 'Main-ED_Codes', 'EDEQ-Score','Lav-Neg_Waardering','Lav-Gebrek_Vertrouwdheid', 'Lav-Alg_Ontevredenheid', 
'SQ48-Hostility','SQ48-Agorafobie','SQ48-Angst','SQ48-Depressie','SQ48-Cognitieve_Complain',
'SQ48-Somatische_Complain', 'SQ48-Sociale_Fobie','SQ48-Vitaliteit_Optimisme','SQ48-Werk_Studie', 
'MHCSF-EmotionWB', 'MHCSF-SocialWB', 'MHCSF-PsychWB'] 
colRename_dict=dict(zip(cols2consider, colNames))
df_adapted_combo=pd.read_csv(fileloc_data+'maskedDAIsy_MainED_Lav_SQ48_MHCSF_Visit1.tsv', sep='\t', decimal=',')
df_adapted_combo['EDtype'] = df_adapted_combo['EDtype'].astype('category')
df_adapted_combo.astype(dict(zip(colsLAV, ['float', 'float','float']))).dtypes
df_adapted_combo.astype(dict(zip(colsSQ48,['float', 'float','float','float', 'float','float','float', 'float','float']))).dtypes
df_adapted_combo.astype(dict(zip(colsMHCSF, ['float', 'float','float']))).dtypes
df_adapted_combo.rename(columns=colRename_dict, inplace=True)
adapted_combo_cols=['Main-Age','Lav-Alg_Ontevredenheid', 'Lav-Gebrek_Vertrouwdheid','Lav-Neg_Waardering',
    'SQ48-Hostility','SQ48-Agorafobie','SQ48-Angst','SQ48-Depressie','SQ48-Cognitieve_Complain',
    'SQ48-Somatische_Complain', 'SQ48-Sociale_Fobie','SQ48-Vitaliteit_Optimisme','SQ48-Werk_Studie', 'MHCSF-EmotionWB', 
    'MHCSF-SocialWB', 'MHCSF-PsychWB']
#df_adapted_combo['ED_Codes'] = df_adapted_combo['EDtype'].cat.codes

In [3]:

df_train_adapted=df_adapted_combo[adapted_combo_cols].loc[df_adapted_combo['Split']=='Train']
YTrain=df_adapted_combo['EDtype'].loc[df_adapted_combo['Split']=='Train'].to_numpy()
df_test_adapted=df_adapted_combo[adapted_combo_cols].loc[df_adapted_combo['Split']=='Test']
YTest=df_adapted_combo['EDtype'].loc[df_adapted_combo['Split']=='Test'].to_numpy()


In [4]:
rf = RandomForestClassifier(criterion="gini")
pipe_RF_search = Pipeline(steps=[('scaler',  StandardScaler()),
            ('RandomForestClassifier', RandomForestClassifier(criterion="gini"))], verbose=0)
rf_param_grid = { "RandomForestClassifier__n_estimators"      : [100, 250, 300, 500],           
           "RandomForestClassifier__max_features"      : [5,7,11,15]}
rf_grid_search = GridSearchCV(pipe_RF_search, rf_param_grid,cv=3, scoring='balanced_accuracy')
rf_grid_search.fit(df_train_adapted.to_numpy(), YTrain)
df_rf_gs = pd.DataFrame(rf_grid_search.cv_results_)[
    ["param_RandomForestClassifier__n_estimators", "param_RandomForestClassifier__max_features","mean_test_score", ]]

df_rf_gs = df_rf_gs.rename(columns={"param_RandomForestClassifier__n_estimators": "n_Trees",
"param_RandomForestClassifier__max_features": "Max_Features", "mean_test_score": "Bal. Acc"})
df_rf_gs.sort_values(by=["Bal. Acc"], ascending=[False]).head(5)

,n_Trees,Max_Features,Bal. Acc
10,300,11,0.369452
8,100,11,0.363645
14,300,15,0.357169
13,250,15,0.355391
1,250,5,0.353801


In [5]:
pipe_KNN_search = Pipeline(steps=[('scaler',  StandardScaler()),
            ('KNeighborsClassifier', KNeighborsClassifier())], verbose=0)
knn_param_grid = {"KNeighborsClassifier__n_neighbors" : [3,5,7],           
           "KNeighborsClassifier__metric": ['minkowski', 'cosine', 'mahalanobis', 'seuclidean']}
knn_grid_search = GridSearchCV(pipe_KNN_search, knn_param_grid,cv=3, scoring='balanced_accuracy')
knn_grid_search.fit(df_train_adapted.to_numpy(), YTrain)
df_knn_gs = pd.DataFrame(knn_grid_search.cv_results_)[
    ["param_KNeighborsClassifier__n_neighbors", "param_KNeighborsClassifier__metric","mean_test_score", ]]

df_knn_gs = df_knn_gs.rename(columns={"param_KNeighborsClassifier__n_neighbors": "K",
"param_KNeighborsClassifier__metric": "Dist Metric", "mean_test_score": "Bal. Acc"})
df_knn_gs.sort_values(by=["Bal. Acc"], ascending=[False]).head(5)

,K,Dist Metric,Bal. Acc
5,7,cosine,0.295743
4,5,cosine,0.267553
3,3,cosine,0.240569
1,5,minkowski,0.236219
2,7,minkowski,0.236029


In [6]:
pipe_LDA_search = Pipeline(steps=[('scaler',  StandardScaler()),
            ('LinearDiscriminantAnalysis', LinearDiscriminantAnalysis())], verbose=0)
lda_param_grid = {"LinearDiscriminantAnalysis__solver": ['svd', 'lsqr', 'eigen']}
lda_grid_search = GridSearchCV(pipe_LDA_search, lda_param_grid,cv=3, scoring='balanced_accuracy')
lda_grid_search.fit(df_train_adapted.to_numpy(), YTrain)
df_lda_gs = pd.DataFrame(lda_grid_search.cv_results_)[
    ["param_LinearDiscriminantAnalysis__solver","mean_test_score", ]]
df_lda_gs = df_lda_gs.rename(columns={
"param_LinearDiscriminantAnalysis__solver": "Solver", "mean_test_score": "Bal. Acc"})
df_lda_gs.sort_values(by=["Bal. Acc"], ascending=[False]).head(5)


,Solver,Bal. Acc
1,lsqr,0.338214
2,eigen,0.338214
0,svd,0.329264


In [7]:
svc_linear=SVC(kernel="linear", random_state=42)
svc_rbf=SVC(kernel='rbf', random_state=42)

lin_param_grid = {'svc_linear__C': [0.001, 0.01, 0.1, 1, 10, 100]}
pipe_lSVM_search = Pipeline(steps=[('scaler', StandardScaler()), ('svc_linear', svc_linear)], verbose=0)
linSVM_grid_search = GridSearchCV(pipe_lSVM_search, lin_param_grid, cv=3, scoring='balanced_accuracy')
linSVM_grid_search.fit(df_train_adapted.to_numpy(), YTrain)
df_linSVM_gs = pd.DataFrame(linSVM_grid_search.cv_results_)[["param_svc_linear__C","mean_test_score", ]]
df_linSVM_gs = df_linSVM_gs.rename(columns={"param_svc_linear__C": "C", "mean_test_score": "Bal. Acc"})
df_linSVM_gs.sort_values(by=["Bal. Acc"], ascending=[False], inplace=True)


rbf_param_grid = {'svc_rbf__C': [0.001, 0.01, 0.1, 1, 10, 100], 
                  'svc_rbf__gamma':[0.0001, 0.001, 0.01, 1, 10, 100]}
pipe_rSVM_search = Pipeline(steps=[('scaler', StandardScaler()), ('svc_rbf', svc_rbf)], verbose=0)
rbfSVM_grid_search = GridSearchCV(pipe_rSVM_search, rbf_param_grid,cv=3, scoring='balanced_accuracy')
rbfSVM_grid_search.fit(df_train_adapted.to_numpy(), YTrain)
df_rbfSVM_gs = pd.DataFrame(rbfSVM_grid_search.cv_results_)[["param_svc_rbf__C","param_svc_rbf__gamma","mean_test_score"]]
df_rbfSVM_gs = df_rbfSVM_gs.rename(columns={"param_svc_rbf__C": "C","param_svc_rbf__gamma": "Gamma", 
                                            "mean_test_score": "Bal. Acc"})
df_rbfSVM_gs.sort_values(by=["Bal. Acc"], ascending=[False], inplace=True)

print(df_linSVM_gs.head(3))
print(df_rbfSVM_gs.head(4))

     C  Bal. Acc
3    1  0.341639
4   10  0.330055
5  100  0.323356
      C   Gamma  Bal. Acc
31  100   0.001  0.322629
26   10    0.01  0.292661
32  100    0.01  0.285359
30  100  0.0001  0.275333


In [10]:
solvers_types = [ "sgd", "wgd", "lbfgs"] 
lgmlvq=LGMLVQ(distance_type="local-adaptive-squared-euclidean")
pipeline = Pipeline(steps=[('scaler', StandardScaler()),('lgmlvq', lgmlvq)])
param_grid = [
    {   "lgmlvq__prototype_n_per_class":[1,2],
        "lgmlvq__relevance_n_components":[5,6,7,10,11,12],
        "lgmlvq__solver_type": solvers_types,
        "lgmlvq__activation_type": ["identity", "swish"],
    },]
#repeated_kfolds = RepeatedStratifiedKFold(n_splits=5, n_repeats=5)
lgmlvq_search = GridSearchCV(pipeline, param_grid, scoring="balanced_accuracy", cv=5,
    n_jobs=4,return_train_score=False)
# The gridsearch object can be fitted to the data.
lgmlvq_search.fit(df_train_adapted.to_numpy(), YTrain)
df_lgmlvq_search = pd.DataFrame(lgmlvq_search.cv_results_)[["param_lgmlvq__prototype_n_per_class","param_lgmlvq__solver_type",
                    "param_relevance_n_components", "param_lgmlvq__activation_type","mean_test_score"]]
df_lgmlvq_search = df_lgmlvq_search.rename(columns={"param_lgmlvq__prototype_n_per_class": "Prot per cls",
                    "param_lgmlvq__solver_type": "Solver", "param_relevance_n_components":'Num Component',
                    "param_lgmlvq__activation_type":'Activation type', "mean_test_score": "Bal. Acc"})
df_lgmlvq_search.sort_values(by=["Bal. Acc"], ascending=[False], inplace=True)
df_lgmlvq_search

KeyError: "['param_relevance_n_components'] not in index"